# Select groups of students for lab or project work
Students are randomly assigned to groups for multiple rotations (labs or sets) such that we maximize the number of different students that a given student is paired with.

In [6]:
import numpy as np
import random

### Change these

In [4]:
nL = 3                    # number of labs (a student will be in this many different groups)
nS = 16                   # numer of students
nominalGroupSize = 3      # nominal number of students in a group
MaxTrials = 10000         # maximim trials to run in attempting to minimize partner overlap

### Code with output below

In [5]:

nG = int(nS/nominalGroupSize) # number of groups
slop  = nS%nominalGroupSize   # extra studnets that dont fit evenly in groups
groupSizes = np.full(nG, nominalGroupSize)
for i in range(1,slop+1):
    groupSizes[-i] += 1

#------------------------

def get_groups():
    
    G = np.full((nS, nL),0)   # students x lab matrix; elements are group number
    
    students = set(range(nS)) # set of student id's
    
    #--------------- fill in groupings for first Lab
    
    k = 0
    for i in range(nG):
        for j in range(groupSizes[i]):
            G[k,0] = i + 1
            k+=1
    
    #---------------
    
    def students_paired_with_student_I(I, icurrent=0):
        paired_with_I = []
        for j in range(jLab+icurrent):      # previous, or jLab+1 to get previous + up to current (current meaning current place in filling lab slots (cols of G))
            for i in range(nS):
                if I==i: 
                    continue
                if G[i,j] == G[I,j]:
                    paired_with_I.append(i)
        return set(paired_with_I)
    
    def students_not_paired_with_student_I(I, icurrent=0):
        paired_with_I = students_paired_with_student_I(I, icurrent)
        not_paired_with_I = students - paired_with_I - {I}
        return not_paired_with_I
    
    #--------------- fill in groupings for subsequent labs
    
    for jLab in range(1,nL):
    
        students_assigned = set()
        students_not_assigned = students.copy()
        
        for i in range(nG):
            
            students_in_current_group = []
            iGrp = i+1
            
            for j in range(groupSizes[i]):
                
                students_available = students_not_assigned.copy()
                if len(students_in_current_group) == 0:
                    iS = random.sample(students_not_assigned,1)[0]
                else:
                    for s in students_in_current_group:
                        notPaired = students_not_paired_with_student_I(s)
                        students_available = students_available.intersection(notPaired)
                    if len(students_available) > 0:
                        iS = random.sample(students_available,1)[0]
                    else:
                        iS = random.sample(students_not_assigned,1)[0]
                    
                students_in_current_group.append(iS)
                students_assigned.add(iS)
                students_not_assigned -= {iS}
                G[iS, jLab] = iGrp
                #print(iGrp, j+1, iS, students_available)
    return G
    
#-------------- count overlaps

def count_redundancy(G):
    redundancy = np.zeros(nS)
    for iS in range(nS):
        partners = []
        for j in range(nL):
            g = G[iS,j]
            for i in range(nS):
                if i==iS:
                    continue
                if G[i,j]==g:
                    partners.append(i)
        redundancy[iS] = len(partners)-len(set(partners))
    return int(np.sum(redundancy))
    
    
#---------------------------

Gbest = get_groups()
rbest = count_redundancy(Gbest)

for rlz in range(MaxTrials):
    if rbest == 0:
        break
    G = get_groups()
    r = count_redundancy(G)
    
    if r < rbest:
        rbest = r
        Gbest = G.copy()
        #print(rlz, rbest)
print("Number of trials =", rlz,'\n')
        
#---------------------------

print("Group numbers for students in given sets (group rotations):\n")
print(f"    {'Student ':12s}", end='')
for i in range(nL):
    print(f'     Set_{i:d}', end='')
print("\n    ----------------------------------------------------------")
for i in range(nS):
    g = G[i,:]
    s = ""
    for j in G[i,:]:
        s += f'{j:10d}'
    print(f'     {i+1:2d}:       {s}')
    
print("\nRedundancy =", rbest)
            
            

Number of trials = 16 

Group numbers for students in given sets (group rotations):

    Student          Set_0     Set_1     Set_2
    ----------------------------------------------------------
      1:                1         5         4
      2:                1         2         1
      3:                1         4         3
      4:                2         3         1
      5:                2         5         5
      6:                2         1         3
      7:                3         3         2
      8:                3         5         3
      9:                3         2         5
     10:                4         1         4
     11:                4         5         2
     12:                4         4         5
     13:                5         3         4
     14:                5         2         2
     15:                5         1         5
     16:                5         4         1

Redundancy = 0
